In [83]:
import pandas as pd
import jieba
import requests
import re
from jieba import analyse
import time
from collections import Counter
import numpy as np
pd.options.display.max_rows=10000
jieba.load_userdict('final.txt')
topic_list=[]
with open ('topic.txt','r',encoding='utf-8') as file:
    to = file.read()
    to = to.split('\n')
    for w in to:
        topic_list.append(w)

topic_list = topic_list[1:]

topic_list.append('ikea')


def crawl_content(id):
    url = 'https://www.dcard.tw/_api/posts/'+str(id)
    web = requests.get(url)
    jsn = web.json()
    if len(jsn)>5:
        if len(jsn['content'])>150 and len(jsn['title'])>3:
            content = jsn['content']
            title = jsn['title']
            topics = jsn['topics']
            output = content+title
            for w in topics:
                output+=(w+'\n')
            for_del = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',output)
            for d in for_del:
                output= output.replace(d,' ')
            return output
        else:
            pass
    else:
        pass



def crawl_forum(ids):
    url = 'https://www.dcard.tw/_api/posts/'+str(ids)
    web = requests.get(url)
    jsn = web.json()
    return jsn['forumAlias']


c = Counter(topic_list)
def find_kw(sentence):
    result = jieba.analyse.extract_tags(sentence,withWeight=True,topK=1000)

    result_df = pd.DataFrame(result)

    result_df.columns=['topic','tfidf']



    result_df['weight'] = result_df['topic'].apply(lambda x : c[x])

    

    result_df['score'] = result_df['tfidf']* (result_df['weight']*1.2)

    result_df_score = result_df.sort_values('score',ascending=False)
    result_df_score = result_df_score.head(10)
    top_score = np.percentile(result_df_score['score'],[0,25,50,85,100])[3]
    result_df_score = result_df_score[result_df_score['score']>max(top_score,0.3)]
#     print(result_df_score)
    
#     result_df_tfidf = result_df.sort_values('tfidf',ascending=False)
#     result_df_tfidf = result_df_tfidf.head(10)
#     top_tfidf = np.percentile(result_df_tfidf['tfidf'],[0,25,50,85,100])[3]
#     b = result_df_tfidf[result_df_tfidf['tfidf']>max(top_tfidf,0.3)]
#     print(result_df_tfidf)
    
#     result_output = pd.concat([a,b])
    all_topic = result_df_score['topic']
    output_list = []
    for topic in all_topic:
        if topic not in output_list:
            output_list.append(topic)
    return output_list

In [41]:
from selenium import webdriver
from bs4 import BeautifulSoup
import random

In [85]:
forums=['music','manicure','2019_ncov','trending','makeup','entertainer','dressup','travel','photography','tvepisode','boutique',]

driver = webdriver.Chrome()
driver.get("https://dcard-crowd-sourcing.dcard.tw/crowd_sourcing/?author=weed")

for i in range(1):
#     times = random.randrange(5,15)
#     time.sleep(times)
    
    find_id = driver.find_element_by_class_name('topic')
    ids = int(find_id.get_attribute('topic_post_id'))
    forum = crawl_forum(ids)
    if forum in forums:
        topic_list=find_kw(crawl_content(ids))
        if len(topic_list)>0:
            topic_input = driver.find_element_by_class_name('topic')
            for w in topic_list:
                topic_input.send_keys(w)
                topic_input.click()
                topic_input.clear()
    #         save = driver.find_element_by_class_name('save_btn1')
    #         save.click()
    #         time.sleep(.3)
    #         next_one = driver.find_element_by_class_name('save_btn2')
    #         next_one.click()
        else:
            driver.refresh()
    else:
        driver.refresh()


In [39]:
find_kw(crowl_content(232992711))

   topic     tfidf  weight      score
0     口罩  0.658229      30  23.696253
59  武漢肺炎  0.035161      99   4.177136


['口罩', '武漢肺炎']

In [5]:
len(crowl_content(232992711))

1035

In [14]:
find_kw(crowl_content(232995397))

   topic     tfidf  weight      score
25   民進黨  0.163764      59  11.758251
0    賴品妤  0.491292       6   4.028593


['民進黨', '賴品妤']

In [15]:
len(crowl_content(232995397))

261

In [31]:
test = pd.DataFrame(c.items())

In [32]:
test[test[1]>5]

,0,1
3,民進黨,59
12,男友,10
17,美甲,29
22,貓,161
25,神魔之塔,7
27,韓國瑜,172
29,乾肌,7
39,煙火攝影,6
42,電腦,8
45,2020總統大選,279


In [77]:
url = 'https://www.dcard.tw/_api/posts/232995397'
web = requests.get(url)
jsn = web.json()

In [78]:
jsn

{'id': 232995397,
 'title': '媽呀帥哥你誰！',
 'content': '圖片來自今天的立法院上班路😆\n\nhttps://i.imgur.com/2aG2g51.jpg\nhttps://i.imgur.com/mSc93R9.jpg\n\n這個愛豆瀏海！\n這個格紋西裝！\n\nhttps://i.imgur.com/1H2wveA.jpg\n\n動態版的短毛賴品妤也好看🤭\nhttps://youtu.be/SJGU6ZH_T9A\n\n如有不妥麻煩大家告訴我修改🙇\n拜託拜託感謝感謝！\n\n======================\n\n阿那個！不記得這誰的人們！\n我給大家講解下嘿~\n這位是民進黨立委賴品妤\n曾任左邊那位林昶佐的助理\n在太陽花學運的時候是太陽花女戰神\n\n前陣子知名的事件是“頭髮拔麥”\n圖片給各位參考一下品妤瀟灑的身姿\nhttps://i.imgur.com/DaBS9EK.jpg\n\n然後然後我不是什麼黨工喔💦\n我只是一個卑微的顏狗\n在此懇求大家不要出征我🙇\nhttps://i.imgur.com/DRRyESN.jpg',
 'excerpt': '圖片來自今天的立法院上班路，這個愛豆瀏海！，這個格紋西裝！，動態版的短毛賴品妤也好看🤭，如有不妥麻煩大家告訴我修改，拜託拜託感謝感謝！，======================，阿那個！不記得這',
 'anonymousSchool': False,
 'anonymousDepartment': True,
 'pinned': False,
 'forumId': '14f573f5-3f02-4821-8e18-f561d7b7db91',
 'replyId': None,
 'createdAt': '2020-02-01T08:22:50.429Z',
 'updatedAt': '2020-02-01T18:49:12.615Z',
 'commentCount': 177,
 'likeCount': 5754,
 'tags': [],
 'topics': ['賴品妤'],
 'supportedReactions': None,
 'withNickname': False,
 'reportReason'

In [82]:
crawl_forum(232995397)

'entertainer'

In [45]:
web = requests.get('https://www.dcard.tw/_api/forums')
jsn = web.json()

In [49]:
forums_list = []
for i in range(len(jsn)):
    forums_list.append(jsn[i]['alias'])

In [52]:
forums_list

['midnightlab',
 'timecapsule',
 'mother',
 'happynewyear',
 'merryxmas',
 'father',
 '5anniversary',
 'bugreport',
 'ntu',
 'nccu',
 'nctu',
 'tku',
 'ndhu',
 'ncku',
 'ntut',
 'fju',
 'ntnu',
 'nknu',
 'ncue',
 'ncu',
 'fcu',
 'nthu',
 'ncnu',
 'thu',
 'nkfust',
 'ntust',
 'nchu',
 'yzu',
 'nuu',
 'ntou',
 'nsysu',
 'ntpu',
 'cpu',
 'ccu',
 'usc',
 'ym',
 'ntua',
 'shu',
 'ntue',
 'tnua',
 'utaipei',
 'pccu',
 'scu',
 'ncyu',
 'nuk',
 'nutc',
 'ntunhs',
 'isu',
 'cgu',
 'tmu',
 'cycu',
 'csmu',
 'wzu',
 'kuas',
 'cgust',
 'ncut',
 'mcu',
 'niu',
 'nfu',
 'nhcue',
 'ttu',
 'ntub',
 'ntcu',
 'nptu',
 'pu',
 'nkmu',
 'cyut',
 'ndu',
 'stust',
 'nkuht',
 'cmu',
 'cjcu',
 'npust',
 'mcut',
 'asia',
 'uch',
 'stu',
 'au',
 'cnu',
 'kmu',
 'must',
 'nttu',
 'tut',
 'nqu',
 'dyu',
 'nhu',
 'ltu',
 'fy',
 'cute',
 'tcust',
 'tcu',
 'chihlee',
 'knu',
 'hk',
 'nutn',
 'chu',
 'takming',
 'just',
 'delete',
 'lhu',
 'yuntech',
 'csu',
 'feu',
 'wfu',
 'ctust',
 'npu',
 'ksu',
 'hwai',
 'ocu',
 

In [53]:
url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100'
web = requests.get(url)
jsn = web.json()


In [59]:
t100_forums = []
for i in range(len(jsn)):
    t100_forums.append(jsn[i]['forumAlias'])

In [61]:
cf = Counter(t100_forums)

In [63]:
cf.most_common()

[('relationship', 14),
 ('entertainer', 13),
 ('makeup', 11),
 ('trending', 8),
 ('2019_ncov', 7),
 ('mood', 7),
 ('funny', 7),
 ('rainbow', 5),
 ('youtuber', 4),
 ('girl', 3),
 ('dressup', 3),
 ('handicrafts', 2),
 ('joke', 2),
 ('talk', 2),
 ('boutique', 2),
 ('house', 1),
 ('fitness', 1),
 ('pet', 1),
 ('buyonline', 1),
 ('job', 1),
 ('vehicle', 1),
 ('studyabroad', 1),
 ('food', 1),
 ('acg', 1),
 ('tvepisode', 1)]

In [64]:
url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100'
web = requests.get(url)
jsn = web.json()

t10000forums_list = []

for i in range(len(jsn)):

    ids = jsn[i]['id']
    t10000forums_list.append(jsn[i]['forumAlias'])
    last = ids
    
for j in range(100):
    url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + str(last)
    web = requests.get(url)
    jsn = web.json()
    if len(jsn)>1:
        for i in range(len(jsn)):
            ids = jsn[i]['id']
            t10000forums_list.append(jsn[i]['forumAlias'])
            last = ids

In [68]:
c_t10000forums = Counter(t10000forums_list)

In [73]:
c_t10000forums.most_common()

[('relationship', 396),
 ('mood', 254),
 ('talk', 162),
 ('2019_ncov', 150),
 ('trending', 149),
 ('horoscopes', 110),
 ('makeup', 96),
 ('rainbow', 80),
 ('girl', 79),
 ('job', 66),
 ('entertainer', 65),
 ('dressup', 60),
 ('funny', 57),
 ('exam', 54),
 ('3c', 53),
 ('travel', 46),
 ('pet', 41),
 ('buyonline', 37),
 ('photography', 35),
 ('vehicle', 34),
 ('game', 33),
 ('acg', 31),
 ('tvepisode', 27),
 ('food', 23),
 ('money', 21),
 ('boutique', 19),
 ('manicure', 18),
 ('apple', 18),
 ('fitness', 17),
 ('music', 17),
 ('studyabroad', 16),
 ('hairsalon', 16),
 ('house', 15),
 ('sneakers', 15),
 ('youtuber', 14),
 ('language', 14),
 ('graduate_school', 14),
 ('military', 13),
 ('handicrafts', 12),
 ('savemoney', 11),
 ('orthodontics', 11),
 ('movie', 10),
 ('facelift', 10),
 ('sport', 10),
 ('novel', 10),
 ('netflix', 9),
 ('tarot', 9),
 ('cvs', 9),
 ('bts', 8),
 ('lesbian', 8),
 ('nursing', 8),
 ('aviation', 8),
 ('literature', 8),
 ('joke', 7),
 ('parentchild', 7),
 ('caligraphy', 7